<a href="https://colab.research.google.com/github/Faraz-Ghani/Machine-Learning-Projects/blob/main/Pokemon-Battle-Prediction/Pokemon_Battles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import the Libararies

In [100]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

Import the datasets

In [125]:
combat = pd.read_csv("/content/Pokemon/combats.csv")
pokemon = pd.read_csv("/content/Pokemon/pokemon.csv")

pokemon.head()

pokemon['Type 2'].fillna('None', inplace=True)  # Fill NaNs in 'Type 2' since many pokemon are single type


In [102]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
types = pokemon[['Type 1', 'Type 2']]  # Collect types of all pokemon
encoded_types = encoder.fit_transform(types) # Use One hot encoding for types
stats = pokemon.iloc[:,4:10].values # Collect stats of all pokemon
legendary = pokemon.iloc[:,11].astype(int).values.reshape(-1, 1) #Check legendary status

In [103]:
pokemon_features = np.hstack([stats, legendary, encoded_types]) # Convert collected data to fetaures

# Create a mapping from Pokémon ID to their features
pokemon_id_to_features = dict(zip(pokemon['#'], pokemon_features))

In [104]:
def combat_data(combat, pokemon_id_to_features):
  features, labels = [],[]

  for _,row in combat.iterrows(): #For battle
    pokemon_a_features = pokemon_id_to_features[row['First_pokemon']] # Get stats of first pokemon
    pokemon_b_features = pokemon_id_to_features[row['Second_pokemon']] # # Get stats of second pokemon
    #Convert both pokemon data to a horizontal stack and then append that horizontal stack to the features
    features.append(np.hstack([pokemon_a_features,pokemon_b_features]))
    if row['Winner'] == row['First_pokemon']:
      labels.append(0) #Return zero if first pokemon is winner
    else:
      labels.append(1) #Return 1 if second pokemon is winner
  # Return both X and Y
  return np.array(features), np.array(labels)


In [105]:
X, y = combat_data(combat, pokemon_id_to_features)

In [106]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [107]:
# Normalization
scaler = StandardScaler()
X_train = scaler.fit_transform((X_train))
X_val = scaler.transform(X_val)

In [108]:
# Define the model
model = Sequential([
    Dense(128,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(64,activation='relu'),
    Dense(1,activation='sigmoid')
])
# Compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [109]:
# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/20
1250/1250 [==============================] - 4s 2ms/step - loss: 0.3479 - accuracy: 0.8626 - val_loss: 0.2774 - val_accuracy: 0.8981
Epoch 2/20
1250/1250 [==============================] - 3s 2ms/step - loss: 0.2205 - accuracy: 0.9184 - val_loss: 0.1856 - val_accuracy: 0.9280
Epoch 3/20
1250/1250 [==============================] - 3s 2ms/step - loss: 0.1446 - accuracy: 0.9441 - val_loss: 0.1438 - val_accuracy: 0.9427
Epoch 4/20
1250/1250 [==============================] - 3s 3ms/step - loss: 0.1077 - accuracy: 0.9585 - val_loss: 0.1264 - val_accuracy: 0.9489
Epoch 5/20
1250/1250 [==============================] - 3s 2ms/step - loss: 0.0886 - accuracy: 0.9657 - val_loss: 0.1196 - val_accuracy: 0.9518
Epoch 6/20
1250/1250 [==============================] - 3s 2ms/step - loss: 0.0750 - accuracy: 0.9710 - val_loss: 0.1061 - val_accuracy: 0.9589
Epoch 7/20
1250/1250 [==============================] - 3s 2ms/step - loss: 0.0635 - accuracy: 0.9764 - val_loss: 0.1060 - val_accuracy:

In [118]:
def predict_winner(Pokemon1,Pokemon2):
  Pokemon1_features = pokemon_id_to_features[Pokemon1]
  Pokemon2_features = pokemon_id_to_features[Pokemon2]
  battle_features = np.hstack([Pokemon1_features,Pokemon2_features])
  battle_features = scaler.transform([battle_features])
  prediction = model.predict(battle_features)
  if prediction < 0.5:
    return Pokemon1
  else:
    return Pokemon2

In [124]:
first_pokemon_id = 427 # Mega Rayquaza
second_pokemon_id = 418  # Latias

predicted_winner = predict_winner(first_pokemon_id, second_pokemon_id)
print(f'The predicted winner between Pokémon {pokemon.iloc[first_pokemon_id-1,1]} and Pokémon {pokemon.iloc[second_pokemon_id-1,1]} is Pokémon {pokemon.iloc[predicted_winner-1,1]}')

1/1 [==============================] - 0s 42ms/step
The predicted winner between Pokémon Mega Rayquaza and Pokémon Latias is Pokémon Mega Rayquaza
